In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import numpy as np
from scipy import stats
import gseapy as gp
from gseapy import barplot, dotplot
import seaborn as sns
import matplotlib.pyplot as plt
import PyComplexHeatmap as pch
from PyComplexHeatmap import *

plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi']=300
plt.rcParams['pdf.fonttype']=42

In [2]:
_gender = 'Male'
mc_type = 'CGN'
ct_type = 'All_CellType'

In [3]:
gene_name_to_id = joblib.load('/home/qzeng/project/aging/240112_DMG_CEF/gene_name_to_id_dict')
gene_id_to_name = joblib.load('/home/qzeng/project/aging/240112_DMG_CEF/gene_id_to_name_dict')

gene_meta_path = '/home/qzeng/project/aging/metadata/gencode.vM22.basic.annotation.gene.flat.tsv.gz'
gene_meta = pd.read_csv(gene_meta_path, index_col ='gene_id', sep='\t')
gene_meta['length'] = gene_meta['end'] - gene_meta['start']
gene_length_dict = gene_meta['length'].to_dict()

In [4]:
_dir = '/home/qzeng/project/aging/240112_DMG_CEF'
mc_all = pd.read_csv(f'{_dir}/result/{_gender}.{mc_type}.DMG.csv', index_col = 0)
mc_all['celltype'] = mc_all['celltype'].apply(lambda x: x.replace('/','').replace(' ','_'))

/home/qz/miniconda3/envs/allcools/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
use_cts= joblib.load('mc_rna_corr_cts')
if ct_type == 'Neuron':
    use_cts = [ct for ct in use_cts if ct.endswith('NN') == False]
elif ct_type == 'NN':
    use_cts = [ct for ct in use_cts if ct.endswith('NN') == True]
if ct_type == 'All_CellType':
    use_cts= joblib.load('mc_rna_corr_cts')

In [19]:
_type = 'DEG'
pvalue_cutoff = 0.05
fc_cutoff = 0

In [20]:
goi = 'Suv39h2'

In [21]:
all_deg_df = []

for ct in use_cts:
    rna_df = pd.read_csv(f"DEGs/{ct}.csv", index_col = 0)
    rna_df['log2(Aged/Young)'] = -rna_df['avg_log2FC']
    
    # also filter for DMGs
    deg_df = rna_df[(rna_df['p_val_adj'] < pvalue_cutoff) & (abs(rna_df['log2(Aged/Young)']) > fc_cutoff)]
    deg_df['celltype'] = ct
    if goi in deg_df.index:
        deg_df = deg_df.loc[goi]
        all_deg_df.append(deg_df)


/home/qz/miniconda3/envs/allcools/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == "__main__":


In [22]:
all_deg_df

[p_val                    0.0
 avg_log2FC          1.178646
 pct.1                  0.016
 pct.2                  0.007
 p_val_adj           0.000325
 log2(Aged/Young)   -1.178646
 celltype             DG_Glut
 Name: Suv39h2, dtype: object]